In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training, datasets, iterators, optimizers, serializers
from chainer import Chain
from chainer.training import extensions
import numpy as np
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X = iris.data
X = X.astype(np.float32)
X[0:5]

array([[ 5.0999999 ,  3.5       ,  1.39999998,  0.2       ],
       [ 4.9000001 ,  3.        ,  1.39999998,  0.2       ],
       [ 4.69999981,  3.20000005,  1.29999995,  0.2       ],
       [ 4.5999999 ,  3.0999999 ,  1.5       ,  0.2       ],
       [ 5.        ,  3.5999999 ,  1.39999998,  0.2       ]], dtype=float32)

In [3]:
Y = iris.target
Y = Y.flatten().astype(np.int32)
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [4]:
train, test = datasets.split_dataset_random(chainer.datasets.TupleDataset(X,Y),100)
train_iter = iterators.SerialIterator(train, batch_size=10)
test_iter = iterators.SerialIterator(test, batch_size=1, repeat=False, shuffle=False)

In [5]:
class IrisModel(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(IrisModel,self).__init__(
            l1=L.Linear(None, n_units),
            l2=L.Linear(None, n_units),
            l3=L.Linear(None, n_out)
        )
    def __call__(self,x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y

In [6]:
model = L.Classifier(IrisModel(100,3))
optimizer = optimizers.Adam()
optimizer.setup(model)

In [7]:
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (30, 'epoch'), out='result')
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))

In [8]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.67609     0.805528              0.46           0.66                      
2           0.819584    0.613432              0.67           0.66                      
3           0.555794    0.515159              0.74           0.74                      
4           0.496598    0.447758              0.76           0.78                      
5           0.419513    0.406831              0.93           0.96                      
6           0.38604     0.375728              0.92           0.94                      
7           0.356966    0.347404              0.89           0.96                      
8           0.342536    0.332876              0.92           0.94                      
9           0.312202    0.299896              0.93           1                         
10          0.289594    0.287598              0.97           0.94                      
11          0.284302    0.258191  